<a href="https://colab.research.google.com/github/ML-HW-SYS/a2/blob/main/4_model_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **4. Convert PyTorch Model to TFLite**

## 4.0 Setup GDrive and Git

### Import GitHub dependencies and mount Google Drive
You should now see your GitHub files under the folder icon to the left. Make sure to include your own token.

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username    
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [ ]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a2-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a2-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a2-{YOUR_HANDLE}"

In [ ]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

### Install required packages

In [ ]:
# Install ONNX
!pip3 install onnx

# Install tensorflow-addons
!pip3 install tensorflow-addons
!pip3 install tensorflow-probability

!pip3 install torchaudio

# Install onnx-tensorflow
# !git clone https://github.com/onnx/onnx-tensorflow.git
# !cd onnx-tensorflow && pip3 install -e .
!pip3 install onnx-tf==1.9.0
!pip install 'keras<3.0.0'
!pip install tensorflow==2.12.0

# [Alert] If there is a popup that tells you restart session.
# No worries! Just click restart on your session, rerun the code blocks above, and you're all set.
# No need to rerun this installment block at all.

### Import code dependencies

In [ ]:
# Import libraries
import sys,os

# Add path to onnx-tensorflow
sys.path.insert(0, os.path.join(PROJECT_ROOT, 'onnx-tensorflow'))
sys.path.insert(0, PROJECT_ROOT)

import onnx
import torch
import torch.nn as nn
import numpy as np
import os
import tensorflow as tf
print(f"TensorFlow Version is: {tf.__version__}")
from onnx_tf.backend import prepare

# Import constants to use constants defined for training
from src.constants import *
import src.data_proc as data_proc

## 4.1 Define the Model


**Replace the torch_path model with the model you created in the last section.** 

You can find the name of your file in `TORCH_DIR` under the folder icon to the left. 

In [ ]:
!ls {TORCH_DIR}

Please use a checkpoint from the cell above and place it in the cell below for the `torch_path`

In [ ]:
from src.networks import TinyConv
import os.path as osp

# Create audio processor
audio_processor = data_proc.AudioProcessor(data_dir=DATASET_DIR) # DATASET_DIR is defined in constants

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a full precision (float32) TinyConv model
model_fp32 = TinyConv(model_settings=audio_processor.model_settings, \
    n_input=1, n_output=audio_processor.num_labels)
model_fp32.to(device)

# TODO: Replace me!
torch_path = osp.join(TORCH_DIR, "tinyconv_float32_init_seed0_90.06%_0.pt")

# Load model
model_fp32.load_state_dict(torch.load(torch_path))
model_fp32

## 4.2 Convert from PyTorch to TFLite
This is the last step before deploying your model to your TinyML device. 

In [ ]:
from src.train_val_test_utils import choose_name
# Choose a path
# You can also define your own path
file_name = choose_name("temp")

### Convert to ONNX

In [ ]:
# Create a new TinyConv model
ONNX_PATH = os.path.join(ONNX_DIR, f'{file_name}.onnx')
TF_PATH = os.path.join(TF_DIR, f'{file_name}')

# Create a random input for model export
dummy_input = torch.autograd.Variable(torch.randn(1, 1960))

# Export model as .onnx file
torch.onnx.export(model_fp32.cpu(), dummy_input, ONNX_PATH, input_names=['input'], output_names=['output'])

# Load onnx model into a tensorflow model
onnx_model = onnx.load(ONNX_PATH)
tf_rep = prepare(onnx_model) 

# Export model as .pb file

tf_rep.export_graph(TF_PATH)

### Convert to TFLite

In [ ]:
FLOAT_MODEL_TFLITE = os.path.join(TFLITE_DIR, f'float_{file_name}.tflite')
MODEL_TFLITE = os.path.join(TFLITE_DIR, f'quant_{file_name}.tflite')

float_converter = tf.lite.TFLiteConverter.from_saved_model(TF_PATH)
float_tflite_model = float_converter.convert()
float_tflite_model_size = open(FLOAT_MODEL_TFLITE, "wb").write(float_tflite_model)
print("Float model is %d bytes" % float_tflite_model_size)

converter = tf.lite.TFLiteConverter.from_saved_model(TF_PATH)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
def representative_dataset_gen():
    for i in range(100):
        data, _ = audio_processor.get_data(
            1, i*1, BACKGROUND_FREQUENCY, BACKGROUND_VOLUME_RANGE, TIME_SHIFT_MS, 'testing')
        flattened_data = np.array(data.flatten(), dtype=np.float32).reshape([1, 1960])
        yield [flattened_data]
converter.representative_dataset = representative_dataset_gen

tflite_model = converter.convert()
tflite_model_size = open(MODEL_TFLITE, "wb").write(tflite_model)
print("Quantized model is %d bytes" % tflite_model_size)

Notice how the quantized model is less than a third of the size of the float model? We'll next see how this reduction in size affects model accuracy. 

### TFLite Model Accuracy

In [ ]:
# Helper function to run inference
def run_tflite_inference_test_set(tflite_model_path, model_type="Float"):
    # Load test data
    test_data, test_labels = audio_processor.get_data(
          -1, 0, BACKGROUND_FREQUENCY, BACKGROUND_VOLUME_RANGE,
          TIME_SHIFT_MS, 'testing')
    test_data = np.expand_dims(test_data, axis=1).astype(np.float32)

    # Initialize the interpreter
    interpreter = tf.lite.Interpreter(tflite_model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    # For quantized models, manually quantize the input data from float to integer
    if model_type == "Quantized":
        input_scale, input_zero_point = input_details["quantization"]
        test_data = test_data / input_scale + input_zero_point
        test_data = test_data.astype(input_details["dtype"])

    # Evaluate the predictions
    correct_predictions = 0
    for i in range(len(test_data)):
        interpreter.set_tensor(input_details["index"], test_data[i])
        interpreter.invoke()
        output = interpreter.get_tensor(output_details["index"])[0]
        top_prediction = output.argmax()
        correct_predictions += (top_prediction == test_labels[i])

    print('%s model accuracy is %f%% (Number of test samples=%d)' % (
        model_type, (correct_predictions * 100) / len(test_data), len(test_data)))

In [ ]:
# Compute float model accuracy
run_tflite_inference_test_set(FLOAT_MODEL_TFLITE)

# Compute quantized model accuracy
run_tflite_inference_test_set(MODEL_TFLITE, model_type='Quantized')

As you can see, quantizing the model hardly affects the model accuracy in this case and makes it much more easily deployable to a tiny machine. 

We will now create the byte version of our model needed for deployment. 

**Replace the name of file in MODEL_TFLITE with the *quant* model you saved earlier in this notebook.**

You can find the name of your file in "{PROJECT_ROOT}/models/tflite_models" under the folder icon to the left. 

### TFLite Micro Conversion

In [ ]:
!apt-get update && apt-get -qq install xxd

# Replace me!
MODEL_TFLITE = TFLITE_DIR + '/quant_temp_0.tflite' 
MODEL_TFLITE_MICRO = MICRO_DIR+'/micro_models.cc'

!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')

!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

You can find the below bites in the "{PROJECT_ROOT}/models/micro_models" folder. If you do not see an output after the next cell, make sure you replaced the file name  of {MODEL_TFLITE} in the cell above with your quant model created earlier in this notebook. 

In [ ]:
!cat {MODEL_TFLITE_MICRO}